Setup the following environment variables in `.env` file:

```bash
OPENAI_API_KEY=
LANGCHAIN_API_KEY=
TRACELOOP_BASE_URL=http://localhost:4318
LANGCHAIN_TRACING_V2=true
```

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

In [ ]:
from traceloop.sdk import Traceloop

Traceloop.init(app_name="testing-openllmetry")

In [43]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [ ]:
retriever.invoke("how to upload a dataset")[0]

In [45]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [46]:
tools = [retriever_tool]

In [62]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_google_vertexai import ChatVertexAI

# model = ChatOpenAI(model="gpt-4o-mini")
model = ChatVertexAI(model="gemini-1.5-flash")

In [63]:
model_with_tools = model.bind_tools(tools)

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="How to monitor performance in LangSmith?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [67]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt)

In [68]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)

In [ ]:
agent_executor.invoke({"input": "hi!"})

In [ ]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})

In [71]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [72]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [73]:
import uuid

session_id = str(uuid.uuid4())


In [ ]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    config={"configurable": {"session_id": session_id}},
)

In [ ]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    config={"configurable": {"session_id": session_id}},
)

In [ ]:
agent_with_chat_history.invoke(
    {"input": "how can langsmith help with testing?"},
    config={"configurable": {"session_id": session_id}},
)